In [1]:
!pip install cubes
from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv
from cubes import Workspace

     |████████████████████████████████| 133kB 2.7MB/s 
     |████████████████████████████████| 92kB 4.0MB/s 
  Created wheel for cubes: filename=cubes-1.1-cp36-none-any.whl size=151591 sha256=e0c05fab7ccf21f2b520ffe82545de965e4d64ab681273a93d7efcff051905d4
  Stored in directory: /root/.cache/pip/wheels/1f/89/09/c361b647a6cc65335af4bc1f27b07abd123727f429819d2590
  Created wheel for expressions: filename=expressions-0.2.3-cp36-none-any.whl size=6445 sha256=3022ab0b23dc8f1c880654758f22ad6f862e172f4bab298f360ba5404aaa69c6
  Stored in directory: /root/.cache/pip/wheels/56/67/97/1a31fbf02237daec61a3d75439d782cf13dfc4af31e8b4e80e
Successfully built cubes expressions


In [3]:
#Question 4.

engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "IBRD_Balance_Sheet__FY2010.csv",
                      table_name="ibrd_balance",
                      fields=[
                          ("category", "string"),
                          ("category_label", "string"),
                          ("subcategory", "string"),
                          ("subcategory_label", "string"),
                          ("line_item", "string"),
                          ("year", "integer"),
                          ("amount", "integer")],
                      create_id=True
                     )

workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")
workspace.import_model("minmax_tutorial_model.json")
cube = workspace.cube("ibrd_balance")
browser = workspace.browser(cube)
result = browser.aggregate(drilldown=["year"])
for record in result:
    print(record)

{'year': 2009, 'amount_sum': 550840, 'record_count': 31, 'min_amount': -1683, 'max_amount': 110040}
{'year': 2010, 'amount_sum': 566020, 'record_count': 31, 'min_amount': -3043, 'max_amount': 128577}


In [39]:
#Question 5 Country-income dataset.
#load the csv file using cubes.
cntry_engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(cntry_engine,
                      "country-income.csv",
                      table_name="OnlineShoppingCube",
                      fields=[
                          ("region", "string"),
                          ("age", "integer"),
                          ("income", "integer"),
                          ("online_shopper", "string")],
                      create_id=True
                     )

#create data cube.
myworkspace = Workspace()
myworkspace.register_default_store("sql", url="sqlite:///data.sqlite")
myworkspace.import_model("region_shopping.json")
cube = myworkspace.cube("OnlineShoppingCube")
browser = myworkspace.browser(cube)
result = browser.aggregate()

#produce aggregate results for:
# 1. the whole data cube

print("Total Income: ",result.summary["total_income"])
print("Average Income: ",result.summary["average_income"])
print("Minimum Income: ",result.summary["minimum_income"])
print("Maximum Income: ",result.summary["maximum_income"])

# 2. results per region

print("Region-wise aggregates are :")
result = browser.aggregate(drilldown=["region"])
for record in result:
  print(record)

# 3. results per online shopping activity
print("Aggregates based on online shopping activity are :")
result = browser.aggregate(drilldown=["online_shopper"])
for record in result:
  print(record)

# 4. results for all people aged between 40 and 50.
import cubes as cubes

print("Aggregates for people aged 40-50 are :")
cuts = [cubes.RangeCut("age", [40], [50])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["age"])
for record in result:
  print(record)




Total Income:  768200
Average Income:  76820.0
Minimum Income:  57600
Maximum Income:  99600
Region-wise aggregates are :
{'region': 'Brazil', 'total_income': 193200, 'average_income': 64400.0, 'minimum_income': 57600, 'maximum_income': 73200}
{'region': 'India', 'total_income': 331200, 'average_income': 82800.0, 'minimum_income': 69600, 'maximum_income': 94800}
{'region': 'USA', 'total_income': 243800, 'average_income': 81266.66666666667, 'minimum_income': 64800, 'maximum_income': 99600}
Aggregates based on online shopping activity are :
{'online_shopper': 'No', 'total_income': 386400, 'average_income': 77280.0, 'minimum_income': 62400, 'maximum_income': 99600}
{'online_shopper': 'Yes', 'total_income': 381800, 'average_income': 76360.0, 'minimum_income': 57600, 'maximum_income': 94800}
Aggregates for people aged 40-50 are :
{'age': 40, 'total_income': 69600, 'average_income': 69600.0, 'minimum_income': 69600, 'maximum_income': 69600}
{'age': 42, 'total_income': 80400, 'average_income'

Part 2: Week 6 Lab.

In [3]:
#Question 6. Support vector machine classifier accuracy.
import gzip
import pickle
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Selecting the training data from the original dataset
f = gzip.open('mnist.pkl.gz', 'rb')
X, y = pickle.load(f, encoding='latin1')[0]
f.close()

# Subsampling
sample_size = 2000
X, y = X[:sample_size], y[:sample_size]

#split the dataset into training and test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#create a SVM classifier.
svm_pipe = make_pipeline(StandardScaler(), SVC())  #using default hyperparameters of SVC, default kernel used is rbf.

#train the model using training set.
svm_pipe.fit(X_train, y_train)

#test the model using testing set.
y_pred = svm_pipe.predict(X_test)

#estimate accuracy of the model.
print("Accuracy of SVM classifier = ",metrics.accuracy_score(y_test, y_pred))


Accuracy of SVM classifier =  0.865


In [15]:
#Question 7.
#Random forest classifier using GridSearchCV to find the best hyperparameters and 5-fold cross-validation.
#n_estimators = {50,100,200}
#max_features = {0.1,0.25}

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#Using GridSearchCV to choose hyperparameters based on 5-fold cross-validation.

#dictionary mapping RFC hyperparameter names to list of values to be used.
gridcv_parameters = {'n_estimators' : [50,100,200], 'max_features' : [0.1,0.25]}

rfc = RandomForestClassifier()
rfc_cv = GridSearchCV(rfc, gridcv_parameters, cv=5)  #for 5-fold cross validation
rfc_cv.fit(X_train, y_train)

print('Best hyperparameter setting: {0}.'.format(rfc_cv.best_estimator_))
print('Accuracy : ', rfc_cv.score(X_test, y_test))

Best hyperparameter setting: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=0.1,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False).
Accuracy :  0.9125


In [14]:
#Question 8.
import numpy as np

X = np.array([[1,2],[3,4]])
y_pred = np.array([9,10])

print(X)

print(y_pred)

print(X.mean(axis=0))

[[1 2]
 [3 4]]
[ 9 10]
[2. 3.]
